## First thesis Experiments
In this notbook, I seek to
- explore the data set I've got
- send some openAI request
- run a small sample to get a performance estimate
- vary parameters, models
- run a large sample, potentially from cluster

In [ ]:
!pip install openai
!pip install pandas
!pip install sklearn

In [1]:
import pandas as pd
import pickle as pkl
import openai
from sklearn.metrics import confusion_matrix
import time
import json

# import large file
with open('../ct_df_posts_single_preds.pkl', 'rb') as f:
    pre_df = pkl.load(f)

# shorten to not murder my computer
# df = pre_df.iloc[:50]
#del pre_df

# retrieve API key
import os
api_key = os.environ.get('OPENAI_API_KEY')
# print(api_key)

In [11]:
# only get the ones without disclosures
pre_df = pre_df[pre_df["has_disclosures"]==False]

# sample 50 ads, 50 non-ads
df = pd.concat([pre_df[pre_df.loc[:,"predicted_disclosure"]==True].sample(60),
                pre_df[pre_df.loc[:,"predicted_disclosure"]==False].sample(60)]).copy()
df.reset_index(drop=True,inplace=True)
del pre_df

### First attempt: Using normal models

In [22]:
# Generate classifications for all samples

completions = {"text-ada-001":[], "text-babbage-001":[], "text-curie-001":[], "text-davinci-002":[]}

for model in ["text-ada-001", "text-babbage-001", "text-curie-001", "text-davinci-002"]:
    i=0
    for txt in df.loc[:,"caption"]:
        i+=1
        if i%10==0: print(f"Counter at {i}")

        # timer to stay within my quota
        if i==34: time.sleep(65)
        if i==68: time.sleep(65)

        prompt = f"Judge whether it is likely that the following caption comes from a post that has been sponsored. Try to aim for a 50:50 distribution between True and False.\n\n Post: 'aaaa meu look de hoje é da @cea_brasil ❤️ patrocinadora e dona do look oficial do Rock in Rio! Aproveitem pra acompanhar o perfil da C&A pra ver todos os lookinhos maravilhosos! Amo muito! ❤️'\n Sponsored (True/False): True\n\n Post: 'side by side ~ sisters ❤️'\n Sponsored (True/False): True\n\n Post:{txt}\n Sponsored (True/False):"

        # Generate a response from openai.
        # Flexibly choose ada/babbage/curie/davinci as engine. For davinci, use text-davinci-002.
        response = openai.Completion.create(
            engine=model,
            prompt=prompt,
            max_tokens=5,
            n=3,
            stop=None,
            temperature=0,
        )

        # Save the response text
        completions[model].append(response["choices"][0]["text"])
    print(model+" done")
    time.sleep(65)

Counter at 10
Counter at 20
Counter at 30
Counter at 40
Counter at 50
Counter at 60
Counter at 70
Counter at 80
Counter at 90
Counter at 100


In [23]:
# Make predictions to booleans
completions_as_boolean = [True if response.__contains__("rue") 
                          else False if response.__contains__("als")
                          else "warning" for response in completions]

# check for bad completions
if "warning" in vanilla_TF: print("Warning")

# obtain confusion matrix
matrix = confusion_matrix(df["has_disclosures"],completions_as_boolean)
matrix

array([[48,  2],
       [19, 31]])

In [6]:
# Example Request OpenAI API
response = openai.Completion.create(
    engine="text-davinci-002",
    prompt="Write a longer essay about what PBL is, how it is applied in different instiutions and discuss advantages and disadvanatages in detail",
    max_tokens=1050,
    n=1,
    stop=None,
    temperature=1,
)

### Using the fine-tune API

In [42]:
df.iloc[:2,:]
# last column predicted disclousere is the output from bert model but only ran on the samples that are not disclosed. Because for the disclosed ones we already know its disloced. But it ran without the hashtags

#he meets supervisor on monday to discuss XAI conference plans
#send some prediction scores tonight

,username,caption,caption_hashtags,tagged_users,subscriber_count,ct_score,counts_disabled,likes,likes_expected,comments,...,setting,has_sponsored_hashtags,has_sponsored_keywords,has_disclosures,caption_tokenized,caption_task,shortcode,followers,dt_year_mon,predicted_disclosure
0,niinasecrets,Comece recomece quantas vezes quiser... Agora ...,[publi],[voucolar],1554141,-2.028338,False,25276,51373,237,...,BR_mega,True,True,True,Comece recomece quantas vezes quiser ... Agora...,Comece recomece quantas vezes quiser ... Agora...,BDGpfWxy_Ov,3548127,2016/03,False
1,niinasecrets,"Genteee, recebi um convite babado! Mês que vem...","[senaibrasilfashion, brasilfashion, senaicetiq...",[agencia_samba],2084430,1.041199,False,53597,51373,284,...,BR_mega,True,True,True,"Genteee , recebi um convite babado ! Mês que v...","Genteee , recebi um convite babado ! Mês que v...",BMPtWbZBPJk,3548127,2016/10,False


In [39]:
# take 10 true and false ones
train_examples = pd.concat([df.iloc[:10],df.iloc[-10:]])

# store them as jsonl like {"text":"some text","label":"True"} \n {...}
train_examples.rename(columns={"caption":"text","has_disclosures":"label"},inplace=True)
train_examples.loc[:,["text","label"]].to_json("train2.jsonl", orient='records', lines=True)

In [1]:
!pip install --upgrade openai

  Using cached openai-0.26.5-py3-none-any.whl
  Using cached openai-0.26.4-py3-none-any.whl


In [2]:
!openai tools fine_tunes.prepare_data -f train.jsonl -q

/bin/bash: line 1: openai: command not found
